In [2]:
from astropy.io import fits
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

# Basic Steps
For a given spatial pixel $p = (x,y)$ with $N$ values of observed line ratios $R_N(p)$ with errors on the line ratios $E_N(p)$ in same units as $R_N$, we use predicted line ratios as function of $A_N(G_0,n)$, to compute differences $\delta_N$ across all $A$:
$${\delta_N^2(p,G_0,n)} = {\left[ R_N(p) - A_N(G_0,n)\over E_N(x)\right] }^2$$
 
The reduced chi-squared for spatial pixel $i$ is then given by

$$\chi^2(p,G_0,n) =  \sum_N {\delta_N^2(p,G_0,n) \over N-1}$$

and the best-fit value of $(G_0,n)$ is the minimum of $\chi^2$  The range of $(G_0,n)$ over which $A_N$ have been computed must be large enough to encompass physically reasonable values in the ISM, typically $10^{-1} < G_0 < 10^7$ in Habing units and $1 < n < 10^7$ cm$^{-3}$

For a spatial map, we do the above computation over all $p$ to produce a spatial maps of $\chi^2$ and best-fit spatial maps of $(G_0,n)$.



In [6]:
class PDRutils:
    def __init__(self):
        self._valid_ratios = []
    

class Measurement:
    def __init__(self,identifier,flux,error):
        self._identifier = identifier
        self._flux      = flux
        self._error     = error
        
    @property
    def flux(self):
        return self._flux
    @property
    def id(self):
        return self._identifier
    @property
    def error(self):
        return self._error
    